In [1]:
# fn='WLASL_v0.3.json'
# proc.CsvCorrector("C:/Users/Administrator/JUPYTER/kursovayz/raw_videos/*",fn,'testing.csv')

In [2]:
import sys
from video_processor import process
df_dict=pd.read_csv("correctec names",dtype=str).to_dict()
proc=process(64,64,30)
res=[]

for key in df_dict.keys():
    if key=='go':
        break
    for index in df_dict[key].keys():
        if df_dict[key][index]=='-1':
            continue
        path1=str(df_dict[key][index])+".mp4"
        path2=str(df_dict[key][index])+".swf"
        vid1=proc.VidConveter(path1,issave=True)
        vid2=proc.VidConveter(path2,issave=True)
        if vid1 == '-1' and vid2=='-1':            
            print("Unexeptable type of video. VID:",df_dict[key][index])
            sys.exit()

In [5]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from train import x_FrameConc,y2numbers,SaveAndPlot,SaveTrainingLog
from video_processor import process,VideoReader
from sklearn.model_selection import train_test_split

In [6]:
w,h,fps=64,64,30

scaler=MinMaxScaler()

limit_class='go'
path='corrected names.csv'

_x,_y=VideoReader(path,limit_class=limit_class)
x,frame_num=x_FrameConc(_x,w,h)
y=y2numbers(_y)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=42,stratify=y)

x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)
x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, MaxPooling3D, Conv3D, Flatten, Dropout

_loss='sparse_categorical_crossentropy'
kernel_size=(3,3,3)

model3d=Sequential()
model3d.add(Conv3D(16,kernel_size,activation='relu',input_shape=(frame_num,w,h,3)))
model3d.add(MaxPooling3D((2,2,2)))
model3d.add(Dropout(0.5))
model3d.add(Conv3D(32,kernel_size,activation='relu'))
model3d.add(Conv3D(32,kernel_size,activation='relu'))
model3d.add(MaxPooling3D((2,2,2)))
model3d.add(Dropout(0.5))
model3d.add(Conv3D(64,kernel_size,activation='relu'))
model3d.add(Conv3D(64,kernel_size,activation='relu'))
model3d.add(MaxPooling3D((2,2,2)))
model3d.add(Dropout(0.5))
model3d.add(Flatten())
model3d.add(Dense(128,activation='relu'))
model3d.add(Dense(6,activation='softmax'))
model3d.compile(optimizer='adam',loss=_loss,metrics=['accuracy'])
model3d.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 137, 62, 62, 16)   1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 68, 31, 31, 16)   0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 68, 31, 31, 16)    0         
                                                                 
 conv3d_1 (Conv3D)           (None, 66, 29, 29, 32)    13856     
                                                                 
 conv3d_2 (Conv3D)           (None, 64, 27, 27, 32)    27680     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 32, 13, 13, 32)   0         
 3D)                                                    

In [11]:
_epochs=30
history=model3d.fit(x_train,y_train,epochs=_epochs,validation_split=0.2)

SaveTrainingLog(model3d,history)
SaveAndPlot(history)

model3d.evaluate(x_test,y_test)

Epoch 1/30
2/2 - 64s - loss: 1.8112 - accuracy: 0.2121 - val_loss: 1.7520 - val_accuracy: 0.0588 - 64s/epoch - 32s/step
Epoch 2/30
2/2 - 58s - loss: 1.6048 - accuracy: 0.2727 - val_loss: 1.7327 - val_accuracy: 0.2353 - 58s/epoch - 29s/step
Epoch 3/30
2/2 - 58s - loss: 1.5331 - accuracy: 0.3030 - val_loss: 1.7516 - val_accuracy: 0.1176 - 58s/epoch - 29s/step
Epoch 4/30
2/2 - 59s - loss: 1.4548 - accuracy: 0.3333 - val_loss: 1.7654 - val_accuracy: 0.0000e+00 - 59s/epoch - 29s/step
Epoch 5/30
2/2 - 58s - loss: 1.4850 - accuracy: 0.3939 - val_loss: 1.7617 - val_accuracy: 0.0588 - 58s/epoch - 29s/step
Epoch 6/30
2/2 - 59s - loss: 1.4806 - accuracy: 0.3939 - val_loss: 1.7429 - val_accuracy: 0.1765 - 59s/epoch - 29s/step
Epoch 7/30
2/2 - 62s - loss: 1.4628 - accuracy: 0.3636 - val_loss: 1.6912 - val_accuracy: 0.2353 - 62s/epoch - 31s/step
Epoch 8/30
2/2 - 61s - loss: 1.3593 - accuracy: 0.3939 - val_loss: 1.6725 - val_accuracy: 0.2941 - 61s/epoch - 31s/step
Epoch 9/30
2/2 - 63s - loss: 1.2512 